# *Vector field computed from a scalar potential*
***

In this example a vector field $\mathbf{B}$ is computed from a scalar field $u$ as $\mathbf{B} = \nabla u$. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import numba
import numba.experimental.jitclass as jitclass

In [ ]:
import pysmsh

Begin by defining a class that computes the scalar potential of a fictional magnetic point charge. 
The resulting magnetic field is a proper magnetic field (in terms of being divergence free) as long as the
position of the charge is excluded from the considered domain.

In [ ]:
@jitclass()
class PointCharge:
    
    charge : numba.float64
    origin : numba.float64[:]
    
    def __init__(self, charge=1.0, origin=(0, 0, 0)): 
        self.charge = charge
        self.origin = np.asarray(origin, dtype=numba.float64)
    
    def value(self, x, y, z):
        
        x0, y0, z0 = self.origin
        
        return self.charge/np.sqrt((x - x0)**2 + (y - y0)**2 + (z - z0)**2) 

Let's create a magnetic field configuration, a so-called bipole field, produced by two charges of opposite signs.

In [ ]:
q1 = PointCharge(origin=(-1.0, 0.0, -0.2))
q2 = PointCharge(origin=( 1.0, 0.0, -0.2), charge=-1.0)

In [ ]:
@jitclass()
class Bipole:

    q1 : PointCharge
    q2 : PointCharge
    
    def __init__(self, q1, q2):
        self.q1 = q1
        self.q2 = q2
    
    def compute(self, x, y, z):
        return self.q1.value(x, y, z) + self.q2.value(x, y, z)

Instantiate the class with the two created charges

In [ ]:
bipole = Bipole(q1, q2)

Create the grid coordinates for the region of interest

In [ ]:
mesh = pysmsh.Mesh.Rectilinear({"x" : np.linspace(-2.0, 2.0, 128),
                                "y" : np.linspace(-1.0, 1.0, 128),
                                "z" : np.linspace( 0.0, 3.0, 128)}, 
                               num_ghost_cells=1)

Now we create a scalar field that will hold the values of the scalar potential at the centers of the cells that constitute the mesh

In [ ]:
potential = pysmsh.Field.Scalar(mesh, "cell_centered")

To compute the values of the potential, we create a compute function that loops over the mesh coordinates

In [ ]:
@numba.njit()
def compute(compobj, field):
    
    xc, yc, zc = field.mesh.centers
    
    for i, x in enumerate(xc):
        for j, y in enumerate(yc):
            for k, z in enumerate(zc):
                
                field.data[i, j, k] = compobj.compute(x, y, z)

In [ ]:
compute(bipole, potential)

We also need to create a field for the resulting vector field

In [ ]:
magnetic_field = pysmsh.Field.Vector(mesh, "face_staggered")

In [ ]:
import pysmsh.difference.staggered

In [ ]:
pysmsh.difference.staggered.gradient(potential, magnetic_field)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

Bx, By, Bz = magnetic_field.data

p = ax.pcolormesh(magnetic_field.mesh.edges.x,
                  magnetic_field.mesh.edges.y,
                  Bz[:, :, 2].T, 
                  cmap='bwr'
                 )

fig.colorbar(p, ax=ax)

ax.set_aspect("equal")

In [ ]:
divB = pysmsh.Field.Scalar(mesh, "cell_centered")

In [ ]:
pysmsh.difference.staggered.divergence(magnetic_field, divB)